In [1]:
import re
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd


options = webdriver.ChromeOptions()
options.add_argument("headless")

In [2]:
driver_path = 'C:/Users/MY/Downloads/chromedriver.exe'

In [3]:
def clean_text(text):
    cleaned_text = re.sub('[a-zA-Z]', '', text)
    cleaned_text = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]',
                          '', cleaned_text)
    cleaned_text = ' '.join(cleaned_text.split())
    return cleaned_text

In [22]:
def biz_khan(query,driver_path):
    domain = '서울신문'
    driver = webdriver.Chrome(executable_path=driver_path, options=options)  # for Windows
    url_domain = 'https://search.naver.com/search.naver?where=news&query='
    news_domain ='%EB%89%B4%EC%8A%A4&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=1081&nso=&is_sug_officeid=0' 
    url = url_domain + query + news_domain
    driver.get(url)
    soup = BeautifulSoup(driver.page_source,'html.parser')
    
    # 기사 url을 list형태로 반환
    news_list = soup.select('#main_pack > section > div > div.group_news > ul')
    url_list = []
    news_list = news_list[0].find_all('li')

    for url in news_list:
        url_list.append(str(url.find_all('a')[0].get('data-url')))
    driver.close()
    
    data_list = []
    head_list = []
    for url in url_list:
        driver = webdriver.Chrome(executable_path=driver_path,options=options)  # for Windows
        time.sleep(2)
        driver.get(url)
        time.sleep(2)
        # try문 안에서 각자 언론사에 맞게 수정
        try:
            soup = BeautifulSoup(driver.page_source,'html.parser')
            head_list.append(clean_text(soup.find('h1', class_ = 'atit2').text))
            data = clean_text(soup.select('#atic_txt1')[0].text)
            data_list.append(data)
        except:
            print('None_type')
        driver.close()
        
    output = pd.DataFrame({'언론사' : [domain]*len(head_list),
                          '제목' :  head_list,
                          '내용' : data_list})
    return output

biz = biz_khan('삼성전자 슬로건',driver_path)

In [8]:
query = '삼성전자 슬로건'
domain = '서울신문'
driver = webdriver.Chrome(executable_path=driver_path, options=options)  # for Windows
url_domain = 'https://search.naver.com/search.naver?where=news&query='
news_domain ='%EB%89%B4%EC%8A%A4&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=1081&nso=&is_sug_officeid=0' 
url = url_domain + query + news_domain
driver.get(url)
soup = BeautifulSoup(driver.page_source,'html.parser')

# 기사 url을 list형태로 반환
news_list = soup.select('#main_pack > section > div > div.group_news > ul')
url_list = []
news_list = news_list[0].find_all('li')

for url in news_list:
    url_list.append(str(url.find_all('a')[0].get('data-url')))
driver.close()

In [17]:
driver = webdriver.Chrome(executable_path=driver_path)  # for Windows
driver.get(url_list[0])

In [15]:
driver.close()

In [20]:
soup = BeautifulSoup(driver.page_source,'html.parser')
soup.select('#atic_txt1')[0].text

'\n美 ‘CES 2018’ 내일 개막 \nAI·IoT 스마트홈·커넥티드카 차세대 디스플레이 경쟁 막올라 구글·아마존 맞서 AI 전면전 \n\n\n\n\n\n\n\n\n\t\t\t      $(window).load(function() {\n\t\t\t         var lightObj = jQuery(\'#gallery784514 a\').lightBox({\n                        imageLoading:\t\t\t\'https://img.seoul.co.kr/img/12_lightbox-ico-loading.gif\',\n                \t\timageBlank:\t\t\t\t\'https://img.seoul.co.kr/img/12_lightbox-blank.gif\',\n\t\t\t\t\t\timageBtnPrev:\t\t\t\'https://img.seoul.co.kr/img/12_pre_i.gif\',\t\n                \t\timageBtnNext:\t\t\t\'https://img.seoul.co.kr/img/12_next_i.gif\',\t\n                 \t\timageBtnClose:\t\t\t\'https://img.seoul.co.kr/img/12_close_i.gif\',\n\t\t\t\t\t\toverlayOpacity:0.6                           \n\t\t\t         });\n\n\t\t\t         $("#expands_784514").click(function(){\n\t\t\t            jQuery(\'#gallery784514 a\').click();\n\t\t\t         });\n\t\t\t      });\n\t\t\t      \n\n▲ LG전자가 ‘CES 2018’ 개막을 앞두고 미국 라스베이거스에 설치한 ‘씽큐’ 옥외 홍보관 앞을 7일 외국인들이 지나가고 있다.라스베이거스 연합뉴스\n\n

In [23]:
biz.to_csv('서울신문.csv',index=None)

In [24]:
data1 = pd.read_csv('서울신문.csv')

In [25]:
data1

,언론사,제목,내용
0,서울신문,원 삼성 씽큐… 스마트 기술 대전,美 ‘ 2018’ 내일 개막 · 스마트홈·커넥티드카 차세대 디스플레이 경쟁 막올라 ...
1,서울신문,‘삼성표 김치’만 326… 기업들은 왜 김장에 팔걷었나,사회봉사의 대명사 ‘김장나눔’ 야쿠르트 아줌마 수천명 김장 유명세 겨울마다 수많은 ...
2,서울신문,백석예술대 경영행정학부의료경영 보건복지부 장관상 2년 연속 수상,보건복지부와 한국건강증진개발원이 주최한 2019년 음주폐해예방의 달 기념식에서 백석...
3,서울신문,전자 최고 화질로 스마트폰 승부수,야심작 ‘3’ 국내외 동시 출시 레드전자의 상징색의 역습이 시작됐다 초기 스마트폰 ...
4,서울신문,이종락의 재계인맥 대해부11 현대가의 ‘큰 어른’ 정몽구 회장과 ‘장손’ 정의선 부회장,정몽구 회장 현대가 실질적 장남 역할일가 챙겨아들 정의선 부회장 경영 최일선에서 그...
5,서울신문,말로 합시다… ‘음성인식 ’ 가전시장 큰 싸움,세계 3대 가전 박람회 개막 삼성 ‘새로운 일상’ 슬로건 걸고 ‘빅스비’ 냉장고 음...
6,서울신문,“실패 덕에 재정비”… 5 오답노트로 6 성공기 쓴다,조준호 본부장 간담회 “개발 단계부터 협력사와 함께” 삼성 갤8 공개 전 선점 총력...
7,서울신문,구글 스마트폰 ‘픽셀’ 등 자체 제작 하드웨어기기 발표,위주 경영 벗어나… 삼성·애플과 정면승부 불가피 안드로이드 운영체제로 스마트폰 생태...
8,서울신문,3 화질 ‘25601440’ 세계 첫 장착 왜,▲ 스마트폰 3 3 화질 ‘25601440’ 세계 첫 장착 왜’화질의 ’라는 슬로건...
9,서울신문,2009 우수기업 우수상품 15개 기업·상품 선정,서울신문은 27일 올해 한국경제를 이끌어 갈 ‘우수기업 우수상품’에 총 15개 기업...
